# Was wordt lijst

Hoofdvraag: welke landelijke typologie (SBB-code, VvN of rVvN) wordt onze standaard die we als input voor Veg2Hab gebruiken en waarom?
- rVvN is het nieuwst, en beschrijft als het goed is de meeste vegetatietypen. gecheckt: als iets een SBB of VvN code heeft, heeft het ook altijd een rVvN code. Andersom niet.
- De opdrachtgever gaf als commentaar terug dat zij vinden dat VvN de standaard moet zijn.

## Was-wordt lijst inladen

In [1]:
import pandas as pd
import re

In [2]:
ww = pd.read_excel('../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx')
ww = ww[['rVvN','VvN','SBB-code', 'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)']].rename(columns={'Habitattypen (blauw: geldige aanvulling; rood: voorstel; zie Toelichting)':'Habtype', 'SBB-code':'SBB'})

/home/jakko/git/veg2hab/.venv/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
ww.shape

(1244, 4)

In [4]:
# Verwijderen van rijen waarbij zowel VvN en SBB missen
ww = ww[~(ww['SBB'].isna() & ww['VvN'].isna())]

In [5]:
ww.shape

(1116, 4)

In [32]:
ww.nunique()

rVvN       908
VvN        797
SBB        771
Habtype    210
dtype: int64

## Volledigheid typologie was-wordt

rVvN lijkt de meest specifieke vegetatietypologie. Alles met een waarde voor SBB of VvN heeft altijd (1 uitzondering, zie beneden) ook een rVvN code, andersom niet. 

Velden met een SBB code hebben altijd ook een habitattype ingevuld, velden met VvN of rVvN niet.

In [6]:
ww.isna().sum()

rVvN         1
VvN        129
SBB        224
Habtype    183
dtype: int64

In [7]:
# Rijen met een waarde voor rVvN hebben NIET altijd een waarde voor VvN of SBB
ww[~ww['rVvN'].isna()].isna().sum()

rVvN         0
VvN        128
SBB        224
Habtype    183
dtype: int64

In [8]:
# Rijen met een waarde voor VvN hebben WEL altijd een waarde voor rVvN, maar niet voor SBB
ww[~ww['VvN'].isna()].isna().sum()

rVvN         0
VvN          0
SBB        224
Habtype    183
dtype: int64

In [9]:
# Rijen met een waarde voor SBB-code hebben WEL (bijna) altijd een waarde voor rVvN, maar niet voor VvN
ww[~ww['SBB'].isna()].isna().sum()

rVvN         1
VvN        129
SBB          0
Habtype      0
dtype: int64

In [10]:
ww[~ww['SBB'].isna() & ww['rVvN'].isna()]

,rVvN,VvN,SBB,Habtype
1216,NaN,NaN,41A3d,H0000


In [11]:
ww[ww.duplicated(keep=False)]

,rVvN,VvN,SBB,Habtype


### VvN rijen splitsen

Sommige rijen voor VvN hebben dubbele codes, soms met `( )` haakjes.

In [12]:
ww.shape

(1116, 4)

In [13]:
ww['VvN'] = ww['VvN'].str.split(',')
ww = ww.explode('VvN')

In [14]:
ww.shape

(1132, 4)

In [15]:
ww['VvN'][~ww['VvN'].isna()] = ww['VvN'][~ww['VvN'].isna()].apply(lambda x: re.sub('[()]', '', str(x)))

In [16]:
ww.to_excel('../data/ww_split.xlsx', index=False, header=True)

#### SBB naar VvN vertalen en vice versa

**Vraag:** Is er een 1-op-1 of veel-op-1 relatie tussen SBB en VvN?

**Antwoord:** Nee, maar wel bij benadering. 

SBB vertalen naar VvN levert in 612 gevallen een unieke match, in 111 gevallen geen code, en in 48 gevallen meerdere VvN codes.

VvN vertalen naar SBB levert in 501 gevallen een unieke macth, in 217 gevallen geen code, en in 105 gevallen meerdere SBB codes.

#### VvN per SBB

In [17]:
number_of_VvN_per_SBB = ww.groupby('SBB')['VvN'].nunique()

In [18]:
number_of_VvN_per_SBB.value_counts()

VvN
1    606
0    111
3     24
2     16
4     11
5      2
6      1
Name: count, dtype: int64

In [29]:
multiple_vvn = number_of_VvN_per_SBB[number_of_VvN_per_SBB > 1].index.to_list()

In [43]:
number_of_Habs_per_SBB = ww.groupby('SBB')['Habtype'].nunique()
number_of_Habs_per_SBB.value_counts()

Habtype
1    749
2     15
3      7
Name: count, dtype: int64

In [44]:
multiple_habs = number_of_Habs_per_SBB[number_of_Habs_per_SBB > 1].index.to_list()

In [47]:
set(multiple_habs).intersection(multiple_vvn)

{'09B2a',
 '11-h',
 '11-j',
 '12B-c',
 '14-c',
 '14D-d',
 '16C-l',
 '17A1',
 '27A2',
 '29A3',
 '31B1',
 '42-d'}

#### SBB per VvN

In [20]:
number_of_SBB_per_VvN = ww.groupby('VvN')['SBB'].nunique()

In [28]:
number_of_SBB_per_VvN.value_counts()

SBB
1    506
0    183
2     75
3     24
5      4
4      4
6      1
Name: count, dtype: int64

In [29]:
number_of_SBB_per_VvN[number_of_SBB_per_VvN == 5]

VvN
08Bd01    5
11Aa02    5
16aB01    5
43Ab1d    5
Name: SBB, dtype: int64